In [ ]:
__author__ = ["Haolin Pan", "Riade Benbaki"]
__version__ = "École Polytechnique, 2020/3/31"
__data__ = "https://github.com/google-research-datasets/Taskmaster/raw/master/TM-1-2019/self-dialogs.json"

In [1]:
# set up
from collections import defaultdict
import numpy as np
import pandas as pd
import requests
from scipy import stats
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
tf.compat.v1.enable_eager_execution() #Should be commented for training with tf1 only
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import metrics

from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score

Using TensorFlow backend.


## Local Modules

Functions of Topic Clustering are show in the demo `Demo_topic_clustering.ipynb` 

The class `Intent_classification` is to form the data set used for the intent detection

The class `Intent_detection` is to realize the models of detecting the  intents of one sentences

In [2]:
from training_notebooks.src.topic_clustering_model import Topic_clustering
from training_notebooks.src.sub_intents_detection import Intent_classifiction
from training_notebooks.src.sub_intents_detection import Intent_detection

Using TensorFlow backend.


# Conversation Demonstration

In [5]:
ID = Intent_detection(None,None,"training_notebooks/trained weigths/intent_detection")




Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


module elmo loaded


In [8]:
TCM = Topic_clustering("TM-1-2019/self-dialogs.json","training_notebooks/trained weigths/topic_clustering")

module use loaded
module elmo loaded
module nnlm loaded
Gnews Swivel loaded
Loaded weights for model elmo
Loaded weights for model use


In [7]:
class Demo_Conversation(object):
    def __init__(self, model_cl, model_id):
        self.model_cl = model_cl
        self.model_id = model_id
        self.classes_arr = ['auto', 'coffee', 'movie', 'non-opening', 'pizza','restaurant','uber']
        self.opening_response = {"auto" : "\tIt seems you want to repair your vehicle", \
                            "coffee" : "\tIt seems you want to order some coffee", \
                            "movie" : "\tIt seems you want to book movie tickets", \
                            "pizza" : "\tIt seems you want to order some pizza",\
                            "restaurant": "\tIt seems you want to book a table",\
                            "uber": "\tIt seems you want to take ride"}

    def inConversationDetected(self, embed = "elmo"):
        
        print("Conversation Test")
        print("Still working on it, not completed")
        print("Input \'~\' to stop")
        print(128 * "=")
        print("\tHello, what can I do for you?")
        print(128 * "-")
        userInput = ""
        current_topic = "non-opening";
        pre_topic = "non-opening"
        userInput = input()
        prediction = self.model_cl.prediction(embed, [userInput])[0]
        current_topic = prediction
        
        while current_topic == "non-opening":
            print("\t Sorry I can't understand what you mean")
            print(128 * "-")
            prediction = self.model_cl.prediction(embed, [userInput])[0]
            current_topic = prediction
            userInput = input()
        
        pre_topic = current_topic
        print("\ttopic predicted: ", current_topic)
        print("\tentering the scenario ", current_topic)
        print(self.opening_response[current_topic])
        
        
        while (userInput != "~"):  
            prediction = self.model_cl.prediction(embed, [userInput])[0]
            if prediction == "non-opening" or prediction == current_topic:
                
                pred_intents = self.model_id.predict( userInput , current_topic )
                sub_topic_array = list( self.model_id.topic2sub_topic[t].keys() )
                for st in sub_topic_array:
                    if pred_intents[ self.model_id.topic2sub_topic[t][st] - 1] > 0.4:
                        intent.append(st)

                if len(intent) > 0:
                    print("\tDetected intents: ", end =" ")
                    for i in intent:
                        print(i, end  = " ")
                else:
                    print("\tNo intent detected") 
                userInput = input()
                continue
            
            print(128 * "-")
            current_topic = prediction
            print("\ttopic predicted: ", current_topic)
            print("\tentering the scenario ", current_topic)
            print(self.opening_response[current_topic])
            userInput = input()
        
        print("Goodbye, thank you for your attention")

In [9]:
DC = Demo_Conversation(TCM, ID)

In [10]:
DC.inConversationDetected()

Conversation Test
Still working on it, not completed
Input '~' to stop
	Hello, what can I do for you?
--------------------------------------------------------------------------------------------------------------------------------


KeyboardInterrupt: 